## Use deployed prompt flow endpoint, in this case it is a Bing search

In [21]:
import urllib.request
import json
import os
import ssl

from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH = os.getenv("AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH","").strip()
# Replace this with the primary/secondary key or AMLToken for the endpoint
AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH_KEY = os.getenv("AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH_KEY","").strip()
 
def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

def bing_search_endpoint(user_question):
    # Request data goes here
    # The example below assumes JSON formatting which may be updated
    # depending on the format your endpoint expects.
    # More information can be found here:
    # https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
    data = {"question": user_question}
    # data = {"questiont":"Sam Altman join Microsoft and return back to OpenAI as of 11/22/2023?"}
    # data = {"question":"Who is China's president as of 11/22/2023?"}

    body = str.encode(json.dumps(data))

    url = AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH
    # Replace this with the primary/secondary key or AMLToken for the endpoint
    api_key = AAG_PROMPT_FLOW_ENDPOINT_BING_SEARCH_KEY
    if not api_key:
        raise Exception("A key should be provided to invoke the endpoint")

    # The azureml-model-deployment header will force the request to go to a specific deployment.
    # Remove this header to have the request observe the endpoint traffic rules
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ api_key), 'azureml-model-deployment': 'yongpfbingsearchlangchain' }

    req = urllib.request.Request(url, body, headers)

    try:
        response = urllib.request.urlopen(req)

        result = response.read()
        # print(result)
        return result
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))

        # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))

# result = bing_search_endpoint("Who is China's president as of 11/22/2023?")

#### Setup Azure OpenAI

In [22]:
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

openai.api_type = "azure"

AZURE_OPENAI_API_VERSION = os.getenv("AAG_AZURE_OPENAI_API_VERSION")
openai.api_version = AZURE_OPENAI_API_VERSION

AZURE_OPENAI_API_KEY = os.getenv("AAG_AZURE_OPENAI_API_KEY").strip()
assert AZURE_OPENAI_API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = AZURE_OPENAI_API_KEY

AZURE_OPENAI_ENDPOINT = os.getenv("AAG_AZURE_OPENAI_ENDPOINT","").strip()
assert AZURE_OPENAI_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
openai.api_base = AZURE_OPENAI_ENDPOINT

# Deployment for Chat
# DEPLOYMENT_NAME_CHAT = os.getenv('DEPLOYMENT_NAME_CHAT')
DEPLOYMENT_NAME_CHAT = os.getenv('AAG_DEPLOYMENT_NAME_CHAT_16K')

# Deployment for embedding
DEPLOYMENT_NAME_EMBEDDING = os.getenv("AAG_DEPLOYMENT_NAME_EMBEDDING")
model: str = DEPLOYMENT_NAME_EMBEDDING

# Deployment for embedding
BING_SUBSCRIPTION_KEY = os.getenv("BING_SUBSCRIPTION_KEY")

In [23]:
from langchain.docstore.document import Document
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains.summarize import load_summarize_chain

from langchain.chat_models import AzureChatOpenAI

# Construct Azure OpenAI
llm = AzureChatOpenAI(
    openai_api_type = "azure",
    api_version = AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT,    
    api_key = AZURE_OPENAI_API_KEY,
    azure_deployment = DEPLOYMENT_NAME_CHAT,
    temperature=0)

def generate_response(txt):
    # Split text
    text_splitter = CharacterTextSplitter()
    texts = text_splitter.split_text(txt)
    # Create multiple documents
    docs = [Document(page_content=t) for t in texts]
    # Text summarization
    chain = load_summarize_chain(llm, chain_type='stuff')
    return chain.run(docs)

#### Main Bot loop

In [24]:
import json

def chat_with_bot():
    while True:
        user_input = input("You: ")
        print("Me:", user_input)
        if user_input.lower() == 'quit':
            print("Exiting conversation.")
            break

        response_bytes = bing_search_endpoint(user_input)

        # Decode the bytes-like object to a string using UTF-8 encoding
        response_string = response_bytes.decode('utf-8')

        # print("Response String:", response_string)  # Print the response for debugging purposes

        try:
            # Parse the JSON data
            parsed_json = json.loads(response_string)
            
            # Access specific keys or values in the parsed JSON
            output_prompt = parsed_json['output_prompt']

            # Print the output prompt
            # print(output_prompt)

            bot_response = generate_response(response_string)
            print("Bot:", bot_response)

        except json.JSONDecodeError as e:
            print("JSON Decode Error:", e)
            print("Invalid JSON received:", response_string)
            # Handle the error or debug accordingly

# Start the conversation
print("Start chatting with the bot ('quit' to exit):")
chat_with_bot()


Start chatting with the bot ('quit' to exit):
Me: Who is China's president as of 11/22/2023?
Bot: Chinese President Xi Jinping has arrived in the United States for his first visit in six years. US President Joe Biden aims to have productive bilateral talks with Xi Jinping. The two leaders previously met at the Asia-Pacific Economic Cooperation summit in Woodside, California. Xi Jinping also met with executives in San Francisco. The 2023 Chinese presidential election was held on March 10, 2023, where Xi Jinping was re-elected as President.
Me: Sam Altman join Microsoft and return back to OpenAI as of 11/23/2023?
Bot: Sam Altman has returned as CEO of OpenAI just days after being ousted from the position. This follows an employee backlash and turmoil within the company. Microsoft has invested $13 billion into OpenAI and Altman is also joining Microsoft. Emmett Shear has been named interim CEO of OpenAI. Talks are ongoing about the future of the company.
Me: quit
Exiting conversation.
